In [ ]:
%%writefile requirements.txt
torch==2.5.0
torchvision==0.20.0
onnx==1.17.0
onnxruntime-gpu==1.19.2
tqdm==4.67.1
albumentations==1.4.20
pandas==2.2.3
seaborn==0.13.2

Writing requirements.txt


In [ ]:

!cat requirements.txt



torch==2.5.0
torchvision==0.20.0
onnx==1.17.0
onnxruntime-gpu==1.19.2          
tqdm==4.67.1
albumentations==1.4.20           
pandas==2.2.3
seaborn==0.13.2                  


In [6]:
!mkdir -p docs
import torch

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory_gb = round(torch.cuda.get_device_properties(0).total_memory / (1024**3), 1)
    status = "GPU доступен"
else:
    gpu_name = "CPU"
    gpu_memory_gb = "—"
    status = "GPU недоступен, используется CPU"

print(f"Статус: {status}")
print(f"GPU: {gpu_name}")
print(f"VRAM: {gpu_memory_gb} GB")


Статус: GPU доступен
GPU: Tesla T4
VRAM: 14.7 GB


In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm.auto import tqdm


if torch.cuda.is_available():
    torch.cuda.empty_cache()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Используемое устройство:", device)


transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

batch_size = 16
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True,  num_workers=2, pin_memory=True)
testloader  = DataLoader(testset,  batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)


model = torchvision.models.efficientnet_b0(weights='DEFAULT')


num_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_features, 10)

model = model.to(device)

print("Модель загружена:", model.__class__.__name__)
print("Количество параметров:", sum(p.numel() for p in model.parameters() if p.requires_grad))

Используемое устройство: cuda
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:00<00:00, 205MB/s]

Модель загружена: EfficientNet
Количество параметров: 4020358


In [13]:
from torch.cuda.amp import autocast, GradScaler

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=5e-4, weight_decay=1e-4)


scaler = GradScaler(enabled=torch.cuda.is_available())

def get_accuracy(model, loader):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    return 100. * correct / total


for epoch in range(5):
    model.train()
    running_loss = 0.0

    for images, labels in tqdm(trainloader, desc=f"Epoch {epoch+1}/5"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        with autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)

        if torch.cuda.is_available():
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()

        running_loss += loss.item()


    train_acc = get_accuracy(model, trainloader)
    test_acc  = get_accuracy(model, testloader)

    print(f'Epoch {epoch+1:2d} | Loss: {running_loss/len(trainloader):.4f} | '
          f'Train acc: {train_acc:.2f}% | Test acc: {test_acc:.2f}%')

print("\nОбучение завершено!")

/tmp/ipython-input-620159926.py:7: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(enabled=torch.cuda.is_available())


Epoch 1/5:   0%|          | 0/3125 [00:00<?, ?it/s]

/tmp/ipython-input-620159926.py:31: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch  1 | Loss: 0.4434 | Train acc: 95.22% | Test acc: 92.74%


Epoch 2/5:   0%|          | 0/3125 [00:00<?, ?it/s]

...

Epoch  3 | Loss: 0.1903 | Train acc: 97.35% | Test acc: 93.48%


Epoch 4/5:   0%|          | 0/3125 [00:00<?, ?it/s]

Epoch  4 | Loss: 0.1551 | Train acc: 98.33% | Test acc: 93.86%


Epoch 5/5:   0%|          | 0/3125 [00:00<?, ?it/s]

Epoch  5 | Loss: 0.1297 | Train acc: 98.20% | Test acc: 93.88%

Обучение завершено!


Модель сохранена!
